In [25]:
import sqlite3
import pandas as pd
import google.generativeai as genai
import json
import gradio as gr
genai.configure(api_key="Your API")
context = json.dumps({
    "database_schema" : {
        'table_name' : 'Titanic',
        'columns' : [
            {'name':'pclass', 'type' : 'string'},
            {'name':'survived', 'type' : 'integer'},
            {'name':'name', 'type' : 'string'},
            {'name':'sex', 'type' : 'string'},
            {'name':'age', 'type' : 'float'},
            {'name':'sibsp', 'type' : 'integer'},
            {'name':'parch', 'type' : 'integer'},
            {'name':'ticket', 'type' : 'string'},
            {'name':'fare', 'type' : 'float'},
            {'name':'cabin', 'type' : 'string'},
            {'name':'embarked', 'type' : 'string'},
            {'name':'boat', 'type' : 'string'},
            {'name':'body', 'type' : 'float'},
            {'name':'home_dest', 'type' : 'string'}
        ]    
},
"column_descriptions" : 
    {'Titanic.pclass' : "Passenger class of a person who travelled in Titanic ship. Values found are '1','2','3' ",
    'Titanic.survived' : "Survived persons denoted as 1 and not survived persons denoted as 0 ",
    'Titanic.name' : "Name of the person travelling in the Titanic ship",
    'sex' : "Gender of the person denoted as 'male' and 'female'",
    'age': "age of the person",
    'sibsp' : "number of siblings or spouses",
    'parch' : "number of parents or children",
    'ticket': "ticket number",
    'fare' : "Price of the ticket",
    'cabin' : "cabin reserved for the person",
    'embarked': "Port the person embarked. 'C', 'Q', 'S' are the values of this column",
    'body' : "identification number of the non-survived person found when recovered",
    'home_dest': "Destination of the person where he/she wants to travel to"
    }
}
)
#Provides sql query for the user questions

def ask_qn(user_question):
    
    instructions = f"you are a sql Query agent. Following is the details of the table {context} generate the sql query for the user question '{user_question}'"
    model = genai.GenerativeModel('gemini-2.5-flash', system_instruction= instructions)
    response = model.generate_content(user_question)
    return response.text

#formats the sql query result from llm model

def format_sql_string( query_string ):
    start = query_string.find('sql')+3
    query_intm = query_string[6:]
    end = query_intm.find('```')
    final_sql_query = query_intm[:end].replace('\n',' ').strip()
    return final_sql_query


def result(a,b):
        db_file = 'My_Dataset.db'  # Name your new SQLite database file here
        df = pd.read_excel("C:\\Users\\karthik.s\\Downloads\\Titanic_Dup.xlsx")
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()
        table_name = 'Titanic'  # Replace with your desired table name
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        
        sql_query = format_sql_string(ask_qn(a))
        df_result = pd.read_sql_query(sql_query,conn)
        
        print(df_result)
        table = df_result.to_markdown(index= False)
        b.append((a, table))
        return "",b
    

with gr.Blocks( theme = gr.themes.Soft()) as demo:
    gr.Markdown("Data Fetch Tool")
    chatbot = gr.Chatbot()
    question = gr.Textbox(label= "Prompt",placeholder="Type your question", info = "Type in the prompt")
    #gr.ClearButton(question,scale = 1)
    question.submit(result,[question,chatbot],[question,chatbot])
    
demo.launch()

C:\Users\karthik.s\AppData\Local\Temp\ipykernel_21076\1547531827.py:82: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
